<a href="https://colab.research.google.com/github/pmsk98/Graduation/blob/main/news_sentiment_classification_kobert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import pandas as pd
import os
os.chdir('/content/gdrive/My Drive/뉴스토픽분류/')
os.getcwd()

train =pd.read_csv('samsung_community_train_set.csv')
test=pd.read_csv('samsung_community_test_set.csv')



In [ ]:
for i in range(len(train)):
    train['text'][i] = str(train['text'][i])

In [ ]:
for i in range(len(test)):
    test['text'][i] = str(test['text'][i])

In [ ]:
train.tail()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

from matplotlib import font_manager, rc
sns.countplot(x=train['label'])

In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
#GPU 사용
device = torch.device("cuda:0")

#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

In [ ]:
train_data = []
for q, label in zip(train['text'],train['label']):
    data =[]
    data.append(q)
    data.append(str(label))

    train_data.append(data)

In [ ]:
train['text']

In [ ]:
test_data=[]
for q, label in zip(test['text'],test['label']):
    data_1 =[]
    data_1.append(q)
    data_1.append(str(label))

    test_data.append(data_1)

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# Setting parameters
max_len = 150
batch_size = 24
warmup_ratio = 0.1
num_epochs = 3
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(train_data, 0, 1, tok, max_len, True, False)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

In [ ]:
#epoch 10
for e in range(num_epochs):
    train_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

In [ ]:
import os
os.chdir('/content/gdrive/My Drive/뉴스토픽분류/')
os.getcwd()

path = '/content/gdrive/My Drive/뉴스토픽분류/'
torch.save(model, path + 'new_topic_model.pt') 
torch.save(model.state_dict(), '7emotions_model_state_dict.pt')  # 모델 객체의 state_dict 저장

torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict()
}, '7emotions_all.tar')  

In [ ]:
#모델 불러오기
import os
os.chdir('/content/gdrive/My Drive/뉴스토픽분류/')

model1 = torch.load('new_topic_model.pt')  # 전체 모델을 통째로 불러옴, 클래스 선언 필수
model1.load_state_dict(torch.load('7emotions_model_state_dict.pt'))  # state_dict를 불러 온 후, 모델에 저장

checkpoint = torch.load('7emotions_all.tar')   # dict 불러오기
model1.load_state_dict(checkpoint['model'])
optimizer.load_state_dict(checkpoint['optimizer'])

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(test_data):
    another_test = BERTDataset(test_data, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model1(token_ids, valid_length, segment_ids)


        test_eval=[]
        
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()
            test_eval.append(np.argmax(logits))
            for e in len(test_eval):
                submission['topic_idx'][e] = test_eval[e]

        print(test_eval)

In [ ]:
another_test = BERTDataset(test_data, 0, 1, tok, max_len, True, False)
test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
test_eval=[]

for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)

    valid_length= valid_length
    label = label.long().to(device)

    out = model(token_ids, valid_length, segment_ids)


    
        
    for i in out:
        logits=i
        logits = logits.detach().cpu().numpy()
        test_eval.append(np.argmax(logits))

In [ ]:
len(test_eval)

In [ ]:
predict= pd.DataFrame(test_eval)

In [ ]:
test['predict'] = None

for i in range(len(test)):
    test['predict'][i] = predict[0][i]

In [ ]:
test['predict'].value_counts()

In [ ]:
test['label'].value_counts()

In [ ]:
label = test['label']
pred = test['predict']

In [ ]:
label =list(test['label'])
pred=list(test['predict'])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(label,pred)

In [ ]:
stock_price =test[['date','Open','High','Low','Close','Volume','predict','label']]

stock_price=stock_price.drop_duplicates(['date'],keep='first')

stock_price = stock_price.reset_index(drop=True)

In [ ]:
for i in range(len(stock_price)):
    stock_price['predict'] = stock_price['label']

In [ ]:
stock_price=stock_price.drop(['label'],axis=1)

In [ ]:
stock_price['predict'].value_counts()

In [ ]:
dup = test.duplicated(['predict'])

In [ ]:
dup

In [ ]:
stock_price['predict'].value_counts()

In [ ]:
prac1 = test.groupby('date')

prac1['predict'].describe()['top'].value_counts()

In [ ]:
prac1['predict'].describe()

In [ ]:

sent_signal =prac1['predict'].describe()['top']

In [ ]:
sent_signal.value_counts()

In [ ]:
stock_price['signal'] = None

for i in range(len(stock_price)):
    stock_price['signal'][i] = sent_signal[i]
    

In [ ]:
stock_price.to_csv('samsung_test_2020_community_predict.csv')

In [ ]:
#stock_price

In [ ]:
pwd